# WNTR Model Development Tutorial

WNTR includes capabilities to help build drinking water distribution network models from geospatial data files (e.g., GeoJSON and Shapefile). The following tutorial illustrates how to generate drinking water distribution network models from perfect and imperfect geospatial datasets. In this tutorial, a perfect dataset represents high quality geospatial data from a drinking water utility that can be used to generate a model without additional modification, while an imperfect dataset represents data from a drinking water utility that requires modifications before WNTR can be used to generate a network model.

The following tutorial uses the KY 4 water distribution network model downloaded from the [UKnowledge Water Distribution Systems Research Database](https://uknowledge.uky.edu/wdsrd/). This model is used to create perfect geospatial data which accurately and wholly reflects junctions, tanks, reservoirs, pipes, and pumps in the original model. The imperfect geospatial data was generated by truncating, skewing, and omitting certain aspects of the perfect data.

Note that additional attributes not contained in geospatial data (i.e., controls, patterns, simulation options) are directly added to the model to replicate the original conditions in the KY 4 model.

The following tutorial contains three WaterNetworkModels:
- wn0 is the base network model built from the original EPANET input (INP) file
- wn1 is a network model built from perfect geospatial data
- wn2 is a network model built from imperfect geospatial data

## Imports
Import WNTR and additional Python packages that are needed for the tutorial.
- Geopandas is used to load geospatial data
- NetworkX is used to calculate distances on the network
- Shapely is used to adjust network geometry
- Matplotlib is used to create graphics
- Warnings is used to suppress warning messages for features that will be addressed in future WNTR releases

In [ ]:
import geopandas as gpd
import networkx as nx
from shapely import LineString
import matplotlib.pylab as plt
import warnings
import wntr

In [ ]:
# Suppress warning messages that will be addressed in future WNTR releases
warnings.filterwarnings("ignore", message="driver GeoJSON does not support open option CRS")
warnings.filterwarnings("ignore", message="Legend does not support handles for PatchCollection instances.")

## Coordinate reference system and units
WNTR uses SI (International System) units (length in meters, time in seconds, mass in kilograms), **with the exception of the distance threshold which is in feet to match the coordinate reference system (CRS) of the geospatial data**. See https://usepa.github.io/WNTR/units.html for more details on WNTR units.

In [ ]:
# Define the CRS of the geospatial data, which is set to **EPSG:3547 NAD83 / Kentucky Single Zone (ftUS)** (see https://epsg.io/3547 for more details).  
crs = "EPSG:3547"  # ft

In [ ]:
# Define coordinates used to zoom in on network graphics
zoom_coords = [(4978500, 4982000), (3903000, 3905500)]

# Base water network model, wn0

## Create base network model, wn0, from the INP file
The following section creates a `WaterNetworkModel` object from an INP file.

In [ ]:
# Create a WaterNetworkModel from an EPANET INP file
wn0 = wntr.network.WaterNetworkModel("networks/ky4.inp")

## Simulate hydraulics and calculate metrics for wn0
Calculate pressure and average expected demand (AED) for use in later comparisons with wn1 and wn2. Note that negative pressures are set to 0, since negative pressures are not realistic.

In [ ]:
# Simulate hydraulics using EPANET 
sim = wntr.sim.EpanetSimulator(wn0)
results0 = sim.run_sim()
# Extract pressure and AED
pressure0 = results0.node["pressure"].loc[0, :]
pressure0[pressure0<0] = 0 # remove negative pressure
aed0 = wntr.metrics.average_expected_demand(wn0)

In [ ]:
# Plot pressure and AED
fig, axes = plt.subplots(1,2, figsize=(12,3.5))
ax = wntr.graphics.plot_network(wn0, node_attribute=aed0, node_size=30, title="wn0 average expected demand", show_plot=False, ax=axes[0], node_colorbar_label='AED\n(m$^3$/s)')
ax = wntr.graphics.plot_network(wn0, node_attribute=pressure0, node_size=30, title="wn0 pressure", show_plot=False, ax=axes[1], node_colorbar_label='Pressure\n(m)')

## Create perfect geospatial data
The `write_geojson` method is used to export the `WaterNetworkModel` to GeoJOSN files to create perfect geospatial data.

In [ ]:
# Create GeoJSON files from the WaterNetworkModel
# Check project folder to verify that files were written
wntr.network.io.write_geojson(wn0, "ky4", crs=crs)

# Perfect geospatial data network model, wn1
The following section creates a `WaterNetworkModel` object from perfect geospatial data. Information not included in geospatial data (i.e., controls, patterns, initial status, simulation options) are then added to the model.

## Create water network model, wn1

GeoJSON files are loaded into WNTR using the `read_geojson` function. The GeoJSON files contain complete attributes for junctions, tanks, reservoirs, pipes, and pumps.  

In [ ]:
# Load the perfect geospatial data from GeoJSON files for wn1
geojson_files = {"junctions": "ky4_junctions.geojson",
    "tanks": "ky4_tanks.geojson",
    "reservoirs": "ky4_reservoirs.geojson",
    "pipes": "ky4_pipes.geojson",
    "pumps": "ky4_pumps.geojson"}
# Create a WaterNetworkModel from GeoJSON files for wn1
wn1 = wntr.network.read_geojson(geojson_files)

### Add controls to wn1
Controls are added to the network model using the string format from EPANET with values in SI units.

In [ ]:
# Add controls to open and close pump based upon the tank level
line = "LINK ~@Pump-1 OPEN IF NODE T-3 BELOW  27.6606"  # 90.75 ft
wn1.add_control("Pump1_open", line)

line = "LINK ~@Pump-1 CLOSED IF NODE T-3 ABOVE  32.2326"  # 105.75 ft
wn1.add_control("Pump1_closed", line)

### Add a demand pattern to wn1
Demand patterns are added to the network model using multipliers and the default pattern name.

In [ ]:
# Create the demand multipliers, assign them to the default demand pattern, and add the pattern to the network model  
multipliers = [
    0.33, 0.25, 0.209, 0.209, 0.259, 0.36,
    0.529, 0.91, 1.2, 1.299, 1.34, 1.34,
    1.32, 1.269, 1.25, 1.25, 1.279, 1.37,
    1.519, 1.7, 1.75, 1.669, 0.899, 0.479,
]
default_pattern_name = wn1.options.hydraulic.pattern
wn1.add_pattern(default_pattern_name, multipliers)

### Add pump initial status to wn1
The initial status of the pump named ~@Pump-1 is set to Closed.

In [ ]:
# Set the initial pump status to closed
pump = wn1.get_link("~@Pump-1")
pump.initial_status = "Closed"

## Write the wn1 model to an EPANET INP file

In [ ]:
wntr.network.write_inpfile(wn1, 'wn1.inp')

## Simulate hydraulics and calculate metrics for wn1
Calculate pressure and AED for use in later comparison with wn0 results. Note that negative pressures are set to 0, since negative pressures are not realistic.

In [ ]:
# Simulate hydrailics using EPANET for wn1
sim = wntr.sim.EpanetSimulator(wn1)
results1 = sim.run_sim()

#Extract pressure and AED for wn1
pressure1 = results1.node["pressure"].loc[0, :]
pressure1[pressure1<0] = 0 # remove negative pressure
aed1 = wntr.metrics.average_expected_demand(wn1)

## Compare the base model (wn0) to the model created from perfect geospatial data (wn1)
Compare the number of components and the difference in AED and pressure (wn0 compared to wn1).

In [ ]:
# Print network attributes for comparison
print(f"Base network attributes: {wn0.describe()}")
print(f"Perfect network attributes: {wn1.describe()}")

In [ ]:
# Calculate absolute difference in AED and pressure
aed_diff1 = (aed0 - aed1).abs()
pressure_diff1 = (pressure0 - pressure1).abs()

In [ ]:
# Plot the pressures and AED for wn0 and wn1 and the absolute difference in pressure and AED
fig, axes = plt.subplots(1,2, figsize=(12,3.5))
ax = wntr.graphics.plot_network(wn0, node_attribute=aed0, node_size=30, title="wn0 average expected demand", show_plot=False, ax=axes[0], node_colorbar_label='AED\n(m$^3$/s)')
ax = wntr.graphics.plot_network(wn0, node_attribute=pressure0, node_size=30, title="wn0 pressure", show_plot=False, ax=axes[1], node_colorbar_label='Pressure\n(m)')

fig, axes = plt.subplots(1,2, figsize=(12,3.5))
ax = wntr.graphics.plot_network(wn1, node_attribute=aed1, node_size=30, title="wn1 average expected demand", show_plot=False, ax=axes[0], node_colorbar_label='AED\n(m$^3$/s)')
ax = wntr.graphics.plot_network(wn1, node_attribute=pressure1, node_size=30, title="wn1 pressure", show_plot=False, ax=axes[1], node_colorbar_label='Pressure\n(m)')

fig, axes = plt.subplots(1,2, figsize=(12,3.5))
ax = wntr.graphics.plot_network(wn1, node_attribute=aed_diff1, node_size=30, title="Difference in average expected demand", show_plot=False, ax=axes[0], node_colorbar_label='AED\ndifference\n(m$^3$/s)')
ax = wntr.graphics.plot_network(wn1, node_attribute=pressure_diff1, node_size=30, title="Difference in pressure", show_plot=False, ax=axes[1], node_colorbar_label='Pressure\ndifference\n(m)')

In [ ]:
# Check that AED and pressure differences between networks are small (< 1e-3)
print(f"Average absolute difference in average expected demand: {aed_diff1.mean()} m^3/s")
print(f"Average absolute difference in pressure: {pressure_diff1.mean()} m")
assert (aed_diff1.mean() < 1e-3), "Average expected demand difference is greater that tolerance"
assert (pressure_diff1.mean() < 1e-3), "Pressure difference is greater that tolerance"

# Imperfect geospatial data network model, wn2

The following imperfections are included in the geospatial data
- Junction data does not exist (no elevation, demand, or coordinates)
- Pipe data has endpoints that do not align, the pipe data also does not contain start and end node names
- Pump data does not contain start and end node names

Tank and reservoir data are complete but need to be associated with the nearest node, respectively

## Refine the geospatial data for wn2

### Load imperfect geospatial data, elevation data, and building data 
Elevation data can be obtained from the [USGS National Map](https://apps.nationalmap.gov/downloader/) and building data can be obtained from [OpenStreetMaps Buildings](https://osmbuildings.org/data/).

In [ ]:
# Load the imperfect pipe and pump data and the tank and reservoir data from the example data folder
# The following code will produce RuntimeWarnings (driver GeoJSON does not support open option CRS), but will still produce expected results
disconnected_pipes = gpd.read_file("data/ky4_disconnected_pipes.geojson", crs=crs)
disconnected_pumps = gpd.read_file("data/ky4_disconnected_pumps.geojson", crs=crs)
tanks = gpd.read_file("data/ky4_tanks.geojson", crs=crs)
reservoirs = gpd.read_file("data/ky4_reservoirs.geojson", crs=crs)

disconnected_pipes.set_index("name", inplace=True)
disconnected_pumps.set_index("name", inplace=True)
tanks.set_index("name", inplace=True)
reservoirs.set_index("name", inplace=True)

In [ ]:
# Load additional datasets including elevation and building data
elevation_data_file = 'data/ky4_elevation.tif' 

buildings = gpd.read_file("data/ky4_buildings.geojson", crs=crs)
buildings.to_crs(crs, inplace=True)

In [ ]:
# Plot the disconnected pipes and buildings
# Note that this plot creates a UserWarning regarding the legend, which will not show polygons
# This is a known limitation of geopandas/matplotlib
fig, ax = plt.subplots(figsize=(12,5))
disconnected_pipes.plot(color="b", label='Disconnected pipes', ax=ax)
buildings.plot(label='Buildings', ax=ax)
ax.legend()
tmp = ax.set_title('Buildings and disconnected pipes')
tmp = ax.set_xticks([])
tmp = ax.set_yticks([])
tmp = ax.set_xlim(zoom_coords[0])
tmp = ax.set_ylim(zoom_coords[1])

In [ ]:
# Rename column whose name does not conform to WNTR naming convention (i.e., 'cv' is changed to 'check_valve') 
disconnected_pipes.rename(columns={'cv':'check_valve'}, inplace=True)

# Use `valid_gis_names` to print a list of valid column names
print(wntr.network.io.valid_gis_names())

### Connect pipes and define junctions
The `connect_lines` function is used to connect pipes within a user specified distance threshold (the threshold is in the same units as the CRS).

In [ ]:
# Define distance threshold used to connect adjacent pipes
# Disconnected pipes outside of this threshold will not be connected (i.e., will remain disconnected)
distance_threshold = 100.0 # ft

# Print the number of disconnected 
print('Number of disconnected pipes', disconnected_pipes.shape[0])
# Connect the disconnected pipes using the distance threshold to create new connected pipes and junctions
pipes, junctions = wntr.gis.connect_lines(disconnected_pipes, distance_threshold)
# Print the number of newly created connected pipes
print('Number of connected pipes', pipes.shape[0])

# Plot the disconnected and the newly created connected pipes and junctions
fig, axes = plt.subplots(1,2,figsize=(12,5))
disconnected_pipes.plot(color="b", label='Disconnected pipes', ax=axes[0])
pipes.plot(color="r", label='Connected pipes', ax=axes[1])
junctions.plot(color="k", label='Junctions', ax=axes[1])
axes[0].legend()
tmp = axes[0].set_title('Disconnected pipes')
tmp = axes[0].set_xticks([])
tmp = axes[0].set_yticks([])
tmp = axes[0].set_xlim(zoom_coords[0])
tmp = axes[0].set_ylim(zoom_coords[1])
axes[1].legend()
tmp = axes[1].set_title('Connected pipes')
tmp = axes[1].set_xticks([])
tmp = axes[1].set_yticks([])
tmp = axes[1].set_xlim(zoom_coords[0])
tmp = axes[1].set_ylim(zoom_coords[1])

### Check connectivity of wn2

In [ ]:
# Create a WaterNetworkModel with only junctions and pipes 
gis_data = wntr.gis.WaterNetworkGIS({"junctions": junctions,
                                     "pipes": pipes})
wn2_temp = wntr.network.from_gis(gis_data)
# Convert WaterNetworkModel to a graph
G = wn2_temp.to_graph()

# Check to see if the graph is connected
uG = G.to_undirected()
print(nx.is_connected(uG))
#print(nx.number_connected_components(uG))

assert nx.is_connected(uG)
# This tutorial assumes that the network is connected at this point, some datasets will still not be connected due to missing tanks/pumps/valves or other inaccuracies

In [ ]:
# Assign elevations to junctions using the sample_raster function 
junction_elevations = wntr.gis.sample_raster(junctions, elevation_data_file)
# Create a new column of elevations for the junctions
junctions["elevation"] = junction_elevations
# Print the junction parameters for only the first 5 entries 
print(junctions.head())

### Snap reservoirs and tanks to the nearest junction
The `snap` function is used to snap reservoirs to junctions within a user specified distance threshold (the threshold is in the same units as the CRS).  

In [ ]:
# Define distance threshold used to connect tanks and/or reservoirs to adjacent junctions
distance_threshold = 100.0 # ft

# Snap the reservoirs to the nearest junction using the distance threshold 
snap_reservoirs = wntr.gis.snap(reservoirs, junctions, distance_threshold)
# Print the reservoir parameters for only the first 5 entries (note there is only 1 reservoir in this example) 
print(reservoirs.head())
# Print the junction and snapped distance for each reservoir
print(snap_reservoirs)

# Snap the tanks to the nearest junction using the distance threshold
snap_tanks = wntr.gis.snap(tanks, junctions, distance_threshold)
# Print the tank parameters for only the first 5 entries (note there are only 4 tanks in this example)
print(tanks.head())
# Print the junction and snapped distance for each tank
print(snap_tanks)

### Connect reservoirs and tanks with a pipe
The `add_connector` function defined below is used to add a pipe between each reservoir/tank and the nearest junction so that they are connected to the network.

In [ ]:
# Define function to add pipes to connect the tanks and reservoirs to the network
def add_connector(snap_attribute, pipes):
    for name, row in snap_attribute.iterrows():
        pipe_crs = pipes.crs
        attributes = {'check_valve': 0, 
                      'diameter': 0.3, 
                      'initial_status': 'Open',
                      'length': 1, 
                      'minor_loss': 0,
                      'roughness': 150,
                      'geometry': LineString([row['geometry'], row['geometry']]),
                      'start_node_name': row['node'],
                      'end_node_name': name}
        pipes.loc[name+'_connector'] = attributes
        pipes.set_crs(pipe_crs, inplace=True)
    return pipes

# Add pipe to connect reservoir
pipes = add_connector(snap_reservoirs, pipes)
# Add pipes to connect tanks
pipes = add_connector(snap_tanks, pipes)
# Print pipe parameters for the newly added connector pipes
print(pipes.tail())

### Estimate demands from building size
Estimate demand using the following steps:
1. Estimate building demand from building area, normalized by the total demand in the system
2. Snap building centroids to junctions 
3. Assign a junction to each building

In [ ]:
# Assign the total network demand
# For this tutorial, the total network demand is assumed to be known based upon the wn0 AED
total_demand = aed0.sum()

In [ ]:
# Proportionally distribute total demand to buildings by area
buildings["area"] = buildings.area
total_building_area = buildings["area"].sum()
buildings["base_demand"] = (buildings["area"] / total_building_area)*total_demand

# Plot the base demand associated with each building
fig, ax = plt.subplots(figsize=(12,5))
ax = buildings.plot(column='base_demand', vmin=0, vmax=0.0002, legend=True, zorder=1, ax=ax, legend_kwds={'label':'Base demand (m$^3$/s)'})
ax = pipes.plot(zorder=0, ax=ax, label='Pipes')
tmp = ax.set_title('Demands associated with each building')
tmp = ax.set_xticks([])
tmp = ax.set_yticks([])
tmp = ax.set_xlim(zoom_coords[0])
tmp = ax.set_ylim(zoom_coords[1])

In [ ]:
# Define distance threshold used to snap buildings to adjacent junctions
distance_threshold = 1000.0 # ft

# Find the centroid of each building
building_centroid = buildings.copy()
building_centroid.geometry = buildings.geometry.centroid
# Snap the buildings to the nearest junction using the distance threshold
snap_buildings = wntr.gis.snap(building_centroid, junctions, distance_threshold)
buildings["junction"] = None
buildings.loc[snap_buildings.index, "junction"] = snap_buildings.loc[:, "node"]

# Print the building parameters for only the first 5 entries 
print(buildings.head())
# Print the junction and snapped distance for each building for only the first 5 entries
print(snap_buildings.head())

## Create water network model, wn2

In [ ]:
# Create a WaterNetworkModel for wn2 from GIS files 
# Pumps are not stored in a GIS files and will be added in following section
gis_data = wntr.gis.WaterNetworkGIS({"junctions": junctions,
                                     "tanks": tanks,
                                     "reservoirs": reservoirs,
                                     "pipes": pipes})
wn2 = wntr.network.from_gis(gis_data)

### Add pumps to wn2
Add pumps to the network model using the following steps:
1. Snap disconnected pumps to pipes
2. Break the pipe that is closest to each pump
3. Determine pump flow direction, based on distance to the nearest reservoir
4. Add the pump to the model

In [ ]:
# Define distance threshold used to snap pumps to pipes
distance_threshold = 100.0 # ft 

# Snap disconnected pumps (created in earlier step) to nearest pipe using the distance threshold
snap_pumps = wntr.gis.snap(disconnected_pumps, pipes, distance_threshold)
# Print the disconnected pump parameters for only the first 5 entries (note there are only 2 pumps in this example)
print(disconnected_pumps.head())
# Print the pipe, junction, and snapped distance for each pump 
print(snap_pumps.head())

In [ ]:
# Calculate the distance to the nearest reservoirs (there is only 1 reservoir in KY 4)
length = wn2.query_link_attribute('length')
# Convert WaterNetworkModel to a graph
G = wn2.to_graph(link_weight = length)
uG = G.to_undirected()
distance_to_reservoir = nx.multi_source_dijkstra_path_length(uG, wn2.reservoir_name_list, weight='weight')

In [ ]:
# Break pipes and update the pumps dataframe
pumps = disconnected_pumps.copy()
for pump_name in disconnected_pumps.index:
    nearest_pipe = snap_pumps.loc[pump_name, 'link']
    pipe = wn2.get_link(nearest_pipe)
    # Determine pump start and end nodes based on distance to the nearest reservoir
    distanceA = distance_to_reservoir[pipe.start_node_name]
    distanceB = distance_to_reservoir[pipe.end_node_name]
    start_node_name = pump_name+'A'
    end_node_name = pump_name+'B'
    if distanceA > distanceB:
        start_node_name = pump_name+'B'
        end_node_name = pump_name+'A'
    # Break pipe to add start and end nodes determined in previous step
    wn2 = wntr.morph.break_pipe(wn2, nearest_pipe, nearest_pipe+'_pump_connector', start_node_name, end_node_name)
    pumps['start_node_name'] = start_node_name
    pumps['end_node_name'] = end_node_name

# Add pumps to wn2 using the updated pumps dataframe (note that this could be done in the loop above with add_pump)
gis_data = wntr.gis.WaterNetworkGIS({"pumps": pumps})
wn2 = wntr.network.from_gis(gis_data, append=wn2)

### Add controls to wn2
Controls are added to the network model using the string format from EPANET with values in SI units.

In [ ]:
# Add controls to open and close pump based upon the tank level
line = "LINK ~@Pump-1 OPEN IF NODE T-3 BELOW  27.6606"  # 90.75 ft
wn2.add_control("Pump1_open", line)

line = "LINK ~@Pump-1 CLOSED IF NODE T-3 ABOVE  32.2326"  # 105.75 ft
wn2.add_control("Pump1_closed", line)

### Add demand pattern and base demand to wn2
Demand patterns are added to the network model using multipliers and the default pattern name. The base demand was estimated from the building footprint (previous step).

In [ ]:
# Define the demand multipliers, assign them to the default demand pattern, and add the pattern to the network model 
# In this example, to be able to compare the networks (wn0 and wn2) the multipliers are the same for both networks. Users may define different multipliers by changing the values assigned to multipliers variable.   
multipliers = [
    0.33, 0.25, 0.209, 0.209, 0.259, 0.36,
    0.529, 0.91, 1.2, 1.299, 1.34, 1.34,
    1.32, 1.269, 1.25, 1.25, 1.279, 1.37,
    1.519, 1.7, 1.75, 1.669, 0.899, 0.479,
]
default_pattern_name = wn2.options.hydraulic.pattern
wn2.add_pattern(default_pattern_name, multipliers)

# Building centroids were snapped to the nearest junction in previous step
# Add building demands to snapped junction
category = None
for i, row in buildings.iterrows():
    junction_name = buildings.loc[i, "junction"]
    if junction_name is None:
        continue
    base_demand = buildings.loc[i, "base_demand"]
    junction = wn2.get_node(junction_name)
    junction.demand_timeseries_list.append((base_demand, default_pattern_name, category))

### Add pump initial status to wn2
The initial status of the pump named ~@Pump-1 is set to Closed.

In [ ]:
# Set the initial pump status to closed
pump = wn2.get_link("~@Pump-1")
pump.initial_status = "Closed"

## Write the wn2 model to an EPANET INP file

In [ ]:
wntr.network.write_inpfile(wn2, 'wn2.inp')

## Simulate hydraulics and calculate metrics for wn2
Calculate pressure and average expected demand for use in later comparison with wn0 results. Note that negative pressures are set to 0, since negative pressures are not realistic.


In [ ]:
# Simulate hydraulics using EPANET for wn2
sim = wntr.sim.EpanetSimulator(wn2)
results2 = sim.run_sim()

# Extract pressure and AED for wn2
pressure2 = results2.node["pressure"].loc[0, :]
pressure2[pressure2<0] = 0 # remove negative pressure
aed2 = wntr.metrics.average_expected_demand(wn2)

In [ ]:
# Plot data and water network model
fig, ax = plt.subplots(figsize=(12,5))
ax = buildings.plot(column='base_demand', vmin=0, vmax=0.0002, zorder=1, legend=True, legend_kwds={"label":"Demand (m$^3$/s)"}, ax=ax)
ax = disconnected_pipes.plot(zorder=0, ax=ax)
ax.set_xticks([])
ax.set_yticks([])
tmp = ax.set_xlim(zoom_coords[0])
tmp = ax.set_ylim(zoom_coords[1])
tmp = ax.set_title('Imperfect geospatial data\nDisconnected pipes and building demand estimated from area')

fig, ax = plt.subplots(figsize=(12,5))
junctions['aed2'] = aed2
ax = junctions.plot(column='aed2', vmin=0, vmax=0.0002, zorder=1, legend=True, legend_kwds={"label":"Demand (m$^3$/s)"}, ax=ax)
ax = pipes.plot(zorder=0, ax=ax)
ax.set_xticks([])
ax.set_yticks([])
tmp = ax.set_xlim(zoom_coords[0])
tmp = ax.set_ylim(zoom_coords[1])
tmp = ax.set_title('Water network model\nConnected pipes and junction demands')

## Compare the base model (wn0) to the model created from imperfect geospatial data (wn2)
Compare the number of components and the difference in average expected demand and pressure (wn0 compared to wn2).

Note that direct node or link comparisons between wn0 and wn2 will not work because the network models do not share the same link and node names. The difference of the mean is used instead of the mean of the difference.

In [ ]:
# Print wn0 and wn2 network characteristics for comparison
print(f"Base network attributes: {wn0.describe()}")
print(f"Imperfect network attributes: {wn2.describe()}")

In [ ]:
# Calculate absolute difference in mean average expected demand and mean pressure
aed_diff2 = abs(aed0.mean() - aed2.mean())
pressure_diff2 = abs(pressure0.mean() - pressure2.mean())

In [ ]:
# Plot the pressures and AED for wn0 and wn2 
fig, axes = plt.subplots(1,2, figsize=(12,3.5))
ax = wntr.graphics.plot_network(wn0, node_attribute=aed0, node_size=30, title="wn0 average expected demand", show_plot=False, ax=axes[0], node_colorbar_label='AED\n(m$^3$/s)')
ax = wntr.graphics.plot_network(wn0, node_attribute=pressure0, node_size=30, title="wn0 pressure", show_plot=False, ax=axes[1], node_colorbar_label='Pressure\n(m)')

fig, axes = plt.subplots(1,2, figsize=(12,3.5))
ax = wntr.graphics.plot_network(wn2, node_attribute=aed2, node_size=30, title="wn2 average expected demand", show_plot=False, ax=axes[0], node_colorbar_label='AED\n(m$^3$/s)')
ax = wntr.graphics.plot_network(wn2, node_attribute=pressure2, node_size=30, title="wn2 pressure", show_plot=False, ax=axes[1], node_colorbar_label='Pressure\n(m)')

In [ ]:
# Check that AED and pressure difference between wn0 and wn2 networks is small (but higher error than wn0/wn1 comparison)
print(f"Average absolute difference in average expected demand: {aed_diff2} m^3/s")
print(f"Average absolute difference in pressure: {pressure_diff2} m") 
assert (aed_diff2 < 1e-5), "Average expected demand difference is greater that tolerance"
assert (pressure_diff2 < 0.05), "Pressure difference is greater that tolerance"

# Troubleshooting
This tutorial shows how to work with a hypothetical imperfect dataset to create a water network model, however other datasets might require different settings or approaches. 
- The ideal snap thresholds for connecting the different geospatial datasets will likely be different in other cases. It is also important to keep in mind that the unit of the snap thresholds matches the CRS of the geospatial data. In some difficult cases, it might be useful to take an iterative approach using multiple snap thresholds.
- GeoJSON files for a water network model need to have column names that are compatible with WNTR. Details can be found in the [documentation](https://usepa.github.io/WNTR/model_io.html#geojson-files).